In [ ]:
import os
import sys

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
import pandas as pd

sys.path.append('../')
from src.data_handler import DataHandler
from src.helper_functions import baseline_als

sns.set_theme()

In [ ]:
data_folder = os.path.join("C:\\", "Data")
figure_folder = os.path.join("C:\\", "5f12dd6ff78b78000186706b", "images", "ch5")

In [ ]:
cooldown_handler = DataHandler(
    data_folder=data_folder,
    figure_folder=figure_folder,
    measurement_folder="20201217_FirstCooldown",
    copy_measurement_folder_structure=False
)

In [ ]:
cooldown_handler.data_folder_tree()

# Full range

In [ ]:
dff = cooldown_handler.read_lakeshore_data(f"temperature-monitoring01_LN2.xls")

for i in range(2, 19):
    if i >= 16:
        dff = pd.concat([dff, cooldown_handler.read_lakeshore_data(f"temperature-monitoring{i}_LHe.xls")])
    else:
        if i < 10:
            i = f"0{i}"
        dff = pd.concat([dff, cooldown_handler.read_lakeshore_data(f"temperature-monitoring{i}_LN2.xls")])

In [ ]:
dff.info()

In [ ]:
fig, ax = plt.subplots(figsize=(5, 4))

time_in_hours = (dff.index - dff.index[0]) / pd.Timedelta(value=1, unit="d")

sns.lineplot(x=time_in_hours, y="Tip Holder", data=dff, ax=ax, label="Holder")
sns.lineplot(x=time_in_hours, y="Magnet Base", data=dff, ax=ax, label="Magnet")
sns.lineplot(x=time_in_hours, y="Baseplate", data=dff, ax=ax, label="Baseplate")

ax.set_xlabel("Time (in days)")
ax.set_ylabel("Temperature (K)")

#ax.axhspan(ymin=75, ymax=80, alpha=0.3, color="C6", label="LN2 boiling point")
#ax.axhspan(ymin=4, ymax=6, alpha=0.3, color="C4", label="LHe boiling point")

ax.axhline(y=77, alpha=0.7, color="C7", linestyle="dashed")
ax.axhline(y=4.2, alpha=0.7, color="C7", linestyle="dashed")
ax.axhline(y=300, alpha=0.7, color="C7", linestyle="dashed")
ax.text(x=10, y=77, s="77 K", color="C7")
ax.text(x=10, y=4.2, s="4.2 K", color="C7")
ax.text(x=10, y=300, s="300 K", color="C7")

ax.set_yscale("log")

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2), ncol=3, frameon=False)

fig.tight_layout()

cooldown_handler.save_figures(fig, "cooldown", only_pdf=True)

# Hold time

In [ ]:
dfh = cooldown_handler.read_lakeshore_data(f"temperature-monitoring16_LHe.xls")

for i in range(17, 19):
    dfh = pd.concat([dfh, cooldown_handler.read_lakeshore_data(f"temperature-monitoring{i}_LHe.xls")])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
time_in_hours = (dfh.index - dfh.index[0]) / pd.Timedelta(value=1, unit="d")

first_idx = 0
sns.lineplot(x=time_in_hours[first_idx:], y="Tip Holder", data=dfh[first_idx:], ax=ax, label="Holder")
sns.lineplot(x=time_in_hours[first_idx:], y="Magnet Base", data=dfh[first_idx:], ax=ax, label="Magnet")
sns.lineplot(x=time_in_hours[first_idx:], y="Baseplate", data=dfh[first_idx:], ax=ax, label="Baseplate")

ax.set_xlabel("Time (in days)")
ax.set_ylabel("Temperature (K)")

ax.set_yscale("log")

ax.axvspan(xmin=0.15, xmax=2.7, ymin=0.04, ymax=0.08, alpha=0.3, color="C7")
ax.text(x=0.7, y=5.5, s=r"Hold time$\geq$2.5 days", color="C7")

ax.legend()
ax.legend().set_visible(False)

fig.tight_layout()
cooldown_handler.save_figures(fig, "hold_time", only_pdf=True)

# LHe levels

(Insufficient data)

In [ ]:
dfl = pd.read_excel(os.path.join(cooldown_handler.data_folder_path, "LHe_level.xlsx"))

In [ ]:
dfl["Time"] = pd.to_datetime(dfl["Time"], infer_datetime_format=True)
dfl = dfl.set_index("Time", drop=True)
time_in_hours = (dfl.index - dfl.index[0]) / pd.Timedelta(value=1, unit="h")

In [ ]:
sns.lineplot(x=time_in_hours, y="LHe level", data=dfl)

# Temperature heatup

In [ ]:
dfheat = cooldown_handler.read_lakeshore_data(f"temperature-monitoring18_LHe.xls")

for i in range(19, 25):
    if i < 20:
        dfheat = pd.concat([dfheat, cooldown_handler.read_lakeshore_data(f"temperature-monitoring{i}_LHe.xls")])
    else:
        dfheat = pd.concat([dfheat, cooldown_handler.read_lakeshore_data(f"temperature-monitoring{i}_heatup.xls")])

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

dfheat = dfheat[(dfheat.index.day > 25) | (dfheat.index.month == 1)]
time_in_days = (dfheat.index - dfheat.index[0]) / pd.Timedelta(value=1, unit="d")


sns.lineplot(x=time_in_days, y="Tip Holder", data=dfheat, ax=ax, label="Holder")
sns.lineplot(x=time_in_days, y="Magnet Base", data=dfheat, ax=ax, label="Magnet")
sns.lineplot(x=time_in_days, y="Baseplate", data=dfheat, ax=ax, label="Baseplate")

ax.set_xlabel("Time (in days)")
ax.set_ylabel("Temperature (K)")

ax.set_yscale("log")

ax.legend()

ax.axhline(y=300, alpha=0.7, color="C7", linestyle="dashed")
ax.text(x=20, y=300, s="300 K", color="C7")
ax.legend().set_visible(False)

fig.tight_layout()

cooldown_handler.save_figures(fig, "heatup", only_pdf=True)